# Projeto

## Bibliotecas

In [3]:
import os

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

## Arquivos

In [4]:
caminho = '../../machine_learning_i/projeto/dados'
arquivo_principal = 'application_train.csv'
arquivo_oculto = 'application_test_student.csv'
arquivo_metadados = 'HomeCredit_columns_description.csv'

## Input

In [5]:
df = pd.read_csv(f'{caminho}/{arquivo_principal}')

df_oculto = pd.read_csv(f'{caminho}/{arquivo_oculto}')

df_metadados = pd.read_csv(f'{caminho}/{arquivo_metadados}', encoding = 'Windows-1252')

## Variáveis

In [6]:
var_expl = [
    'CNT_CHILDREN', 
    'AMT_INCOME_TOTAL', 
    'DAYS_BIRTH', 
    'DAYS_EMPLOYED',
    'NAME_EDUCATION_TYPE'
]

var_resp = 'TARGET'

## Treino e Teste

In [7]:
df_treino, df_teste = train_test_split(df, test_size = 0.15, random_state = 1)

In [8]:
x_treino = df_treino[var_expl].copy()
y_treino = df_treino[var_resp].copy()

x_teste = df_teste[var_expl].copy()
y_teste = df_teste[var_resp].copy()

x_oculto = df_oculto[var_expl].copy()

## Pré-Processamento

In [9]:
lista_ordenada = [
    'Lower secondary',
    'Secondary / secondary special', 
    'Incomplete higher',
    'Higher education', 
    'Academic degree', 
]

oe = OrdinalEncoder(categories = [lista_ordenada])

oe.fit(x_treino[['NAME_EDUCATION_TYPE']])
x_treino[['NAME_EDUCATION_TYPE']] = oe.transform(x_treino[['NAME_EDUCATION_TYPE']])
x_teste[['NAME_EDUCATION_TYPE']] = oe.transform(x_teste[['NAME_EDUCATION_TYPE']])
x_oculto[['NAME_EDUCATION_TYPE']] = oe.transform(x_oculto[['NAME_EDUCATION_TYPE']])

## Modelo e Métricas

In [49]:
%%time
modelo = LGBMClassifier(
    num_leaves = 4,         # similar às árvores, segundo mais importante otimizar
    max_depth = 2,          # similar às árvores, segundo mais importante otimizar
    subsample = 0.1,        # similar ao random forest, importante definir
    subsample_freq = 5,     # similar ao random forest, importante definir
    colsample_bytree = 0.5, # similar ao random forest, importante definir
    n_estimators = 500,     # similar ao random forest, mais importante otimizar
    learning_rate = 0.01,   # específico dos boostings, mais importante otimizar
    random_state = 1,    
)

modelo.fit(x_treino, y_treino)

y_pred_treino = modelo.predict_proba(x_treino)[:, 1]
y_pred_teste = modelo.predict_proba(x_teste)[:, 1]

CPU times: user 12 s, sys: 1.09 s, total: 13 s
Wall time: 1.95 s


In [50]:
roc_auc_score(y_treino, y_pred_treino)

0.6322802000634336

In [51]:
roc_auc_score(y_teste, y_pred_teste)

0.622496007565556

## Previsão

In [ ]:
y_pred_oculto = modelo.predict_proba(x_oculto)[:, 1]
df_oculto['Y_PRED'] = y_pred_oculto
df_oculto[['SK_ID_CURR', 'Y_PRED']].head()